In [1]:
from elasticsearch import Elasticsearch
import nltk
import math
import sys
import enchant
import operator
d = enchant.Dict("en_US")

In [2]:
es = Elasticsearch(['http://localhost:9200/'])
doc = {
        'size' : 10000,
        'query': {
            'match_all' : {}
       }
   }
res = es.search(index='newsgroup', doc_type='document', body=doc,scroll='1m')

In [3]:
topics = {
        'size' : 20,
        'query': {
            'match_all' : {}
       }
   }
res_topics = es.search(index='newsgroup_topic_modelling', body=topics,scroll='1m')

In [4]:
text = res['hits']['hits'][0]['_source']['doc_text']

In [7]:
topic_words = {}
topic_words_probs = {}
for item in res_topics['hits']['hits']:
    id_ = item['_source']['topic_id']
    topic_words[id_] = item['_source']['top_words']
    topic_words_probs[id_] = item['_source']['word_probs']

In [5]:
def score_sentences(doc, doc_topics , topic_words, topic_words_probs):            
    sentences = doc.split(".")
    sentences_score = {}
    text = nltk.tokenize.word_tokenize(doc)
    for word in text:
        if not d.check(word):
            if len(word) < 4:
                text.remove(word)
    for sentence in sentences:
        score = 0
        words_in_sentence = nltk.tokenize.word_tokenize(sentence)
        for word in words_in_sentence:
            for topic in topic_words:
                if word in topic_words[topic]:
                    score += topic_words_probs[topic][topic_words[topic].index(word)]
        sentences_score[sentence] = score
    return sentences_score

In [8]:
summaries = {}
for item in res['hits']['hits']:
    sentence_score = score_sentences(item['_source']['doc_text'], 
                                    item['_source']['doc_topics'],
                                    topic_words,
                                    topic_words_probs)
    sorted_dict = sorted(sentence_score.items(), key=operator.itemgetter(1))[::-1][:10]
    summary = ""
    for sentence in sorted_dict:
        summary += sentence[0]        
    summaries[item['_source']['doc_id']] = summary

In [12]:
summaries["3"]

'|>The student of "regional killings" alias Davidian (not the Davidian religios sect) writes:|>Greater Armenia would stretch from Karabakh, to the Black Sea, to the|>Mediterranean, so if you use the term "Greater Armenia" use it with care|>It has always been up to the Azeris to end their announced winning of Karabakh |>by removing the Armenians! When the president of Azerbaijan, Elchibey, came to |>power last year, he announced he would be be "swimming in Lake Sevan [in |>Armeniaxn] by July"\tOr have you changed your calendar???|>Well, he was wrong! If Elchibey is going to shell the |>Armenians of Karabakh from Aghdam, his people will pay the price! If Elchibey \t\t\t\t\t\t    ****************|>is going to shell Karabakh from Fizuli his people will pay the price! If \t\t\t\t\t\t    ******************|>Elchibey thinks he can get away with bombing Armenia from the hills of |>Kelbajar, his people will pay the price Real accurate!Ohhhh so swedish RedCross workers do lie they too? What ever

In [13]:
docs_in_elastic = {}
doc_topics_in_elastic = {}
kl_summary_in_elastic = {}
for item in res['hits']['hits']:
    id_ = item['_source']['doc_id']
    docs_in_elastic[id_] = item['_source']['doc_text']
    doc_topics_in_elastic[id_] = item['_source']['doc_topics']
    kl_summary_in_elastic[id_] = item['_source']['kl_summary']

In [20]:
# es.indices.delete(index='newsgroup', ignore=[400, 404])
mappings_newsgroup = {
    'mappings':{
        'document':{
            'properties':{
                'doc_id': {'type': 'text', 'index': 'false'},
                'doc_text': {'type': 'text', 'analyzer': 'english'},
                'doc_topics':{'type': 'text', 'analyzer': 'english'},
                'lda_summary':{'type': 'text', 'analyzer': 'english'},
                'kl_summary':{'type': 'text', 'analyzer': 'english'}
            }
        }
    }
}
es.indices.create(index="newsgroup", body=mappings_newsgroup)

{'acknowledged': True, 'index': 'newsgroup', 'shards_acknowledged': True}

In [15]:
docs = []
for id_ in docs_in_elastic:
    docs.append({
        'doc_id' : id_,
        'doc_text': docs_in_elastic[id_],
        'lda_summary' : summaries[id_],
        'doc_topics' : doc_topics_in_elastic[id_],
        'kl_summary': kl_summary_in_elastic[id_]
    })


In [21]:
for doc in docs:
    es.index(index='newsgroup', doc_type='document', body=doc)